## Deploy Logistic Regression model using SageMaker
boto3 - 
The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services. Using the SDK for Python, you can build applications on top of Amazon S3, Amazon EC2, Amazon DynamoDB, and more.

In [10]:
import sagemaker
from sklearn.model_selection import train_test_split
import  pandas as pd
import boto3

In [11]:
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name

# get the unique name of s3 bucket
bucket = 'aws-sagemaker-fraud-detection-s3'


In [16]:
# publish/upload the data in S3
dataset_path_in_s3 = 'sagemaker/fraud_detection/data'

train_data_path = sess.upload_data(
     path='../data/train.csv', bucket=bucket, key_prefix=dataset_path_in_s3
)

test_data_path = sess.upload_data(
     path='../data/test.csv', bucket=bucket, key_prefix=dataset_path_in_s3
)
print(train_data_path)
print(test_data_path)

s3://aws-sagemaker-fraud-detection-s3/sagemaker/fraud_detection/data/train.csv
s3://aws-sagemaker-fraud-detection-s3/sagemaker/fraud_detection/data/test.csv


### Issue in the following code
The error message ModuleNotFoundError: No module named 's3fs' indicates that the Python package s3fs is not installed in your environment. This package is necessary for accessing S3 buckets using pandas, as it enables reading and writing files from S3 in a way that is compatible with pandas' read_csv function.
Solution:
pip install s3fs

In [8]:
from sagemaker import get_execution_role
from sklearn.preprocessing import StandardScaler
import pandas as pd

# link to the data
train_s3_data = 's3://aws-sagemaker-fraud-detection-s3/sagemaker/fraud_detection/data/train.csv'
test_s3_data = 's3://aws-sagemaker-fraud-detection-s3/sagemaker/fraud_detection/data/test.csv'

# Load your data (from S3, you can use pandas or directly split it)
# df_train = pd.read_csv(train_data_path)
# df_test = pd.read_csv(test_data_path)
df_train = pd.read_csv(train_s3_data)
df_test = pd.read_csv(test_s3_data)

X_train, y_train = df_train.drop(columns=['is_fraud']), df_train['is_fraud']
X_test, y_test = df_test.drop(columns=['is_fraud']), df_test['is_fraud']


### Create a Training Script for Logistic Regression
SageMaker allows you to provide a custom training script using its Script Mode. You can save the following code as train_logistic_regression.py in your local directory.
I have created a python script file in src by the name of sagemaker_logistic_regression.py

### Create a SageMaker Estimator to Train the Model
Now, use the Estimator to initiate training on AWS SageMaker.

In [13]:
# Get execution role
# get_execution_role() only work in sagemaker studio not out of it
# role = get_execution_role()
# Manually Finding the Role in IAM: If get_execution_role() doesn’t work for your environment, you can find the role manually:
# Go to the IAM section in the AWS Console.
# Click on Roles on the left sidebar.
# Find or search for the role that has the required permissions for SageMaker.
# Copy the ARN of that role.
# Example of an ARN for a role:
# Account ID (IAM user): 145023122385
# role = 'arn:aws:iam::145023122385:role/aws-sagemaker-practitioner'
role = 'arn:aws:iam::145023122385:role/aws-sagemaker-practitioner'
# Make sure this role has the necessary permissions (SageMaker, S3, etc.).

In [30]:
pd.read_csv(train_data_path).head()

,cc_num,merchant,category,amt,gender,zip,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,6011382886333463,644,0,-0.326279,1,-0.007694,1.196690,0.432759,-0.290814,201,1378557464,43.727750,-85.046376,0
1,370877495212014,87,4,0.785585,0,-0.751571,-0.462636,0.663378,-0.223335,126,1384774507,36.851523,-80.202303,0
2,3566373869538620,103,2,-0.162638,1,1.440961,-0.614888,-1.144865,-0.290618,366,1380454669,34.765582,-106.874102,0
3,6517217825320610,475,8,-0.434735,1,-0.341744,-1.362873,0.003733,-0.289709,258,1380431862,30.818746,-90.609324,0
4,213125815021702,75,13,-0.378975,1,-1.356629,0.697593,1.149876,-0.292369,356,1388422195,42.311107,-74.938746,0


In [15]:
from sagemaker.sklearn.estimator import SKLearn

# Define the SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point="../src/sagemaker_logistic_regression.py",  # Path to your script
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    framework_version="0.23-1",  # Version of sklearn used
    base_job_name="logistic-regrsion-sklearn-custom"
    # we can also pass the custom parameters as well
    # hyperparamters={
    #     'n_estimators': 100,
    #     'random_state': 0
    # }
    # py_version="py3",
    # output_path=f"s3://{bucket}/sagemaker/fraud_detection/output"
    # use_spot_instances=True,
    # max_wait=7200,
    # max_run=3600
)



In [18]:
# Start the training job
sklearn_estimator.fit({'train': train_data_path, 'test': test_data_path})
# sklearn_estimator.fit({'train': train_data_path}, wait=True)

INFO:sagemaker:Creating training-job with name: logistic-regrsion-sklearn-custom-2024-10-24-20-04-51-519


2024-10-24 20:01:41 Starting - Starting the training job......
2024-10-24 20:02:48 Downloading - Downloading input data...
2024-10-24 20:03:14 Downloading - Downloading the training image......
2024-10-24 20:04:10 Training - Training image download completed. Training in progress.
2024-10-24 20:04:10 Uploading - Uploading generated training model2024-10-24 20:04:04,917 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-10-24 20:04:04,920 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-24 20:04:04,960 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-10-24 20:04:05,124 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-24 20:04:05,136 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-24 20:04:05,152 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-24 20

In [20]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
     TrainingJobName=sklearn_estimator.latest_training_job.name
)['ModelArtifacts']['S3ModelArtifacts']

print('Model Artifacts Value')
print(artifact)
# the name and the location of the model


2024-10-24 20:04:23 Starting - Preparing the instances for training
2024-10-24 20:04:23 Downloading - Downloading the training image
2024-10-24 20:04:23 Training - Training image download completed. Training in progress.
2024-10-24 20:04:23 Uploading - Uploading generated training model
2024-10-24 20:04:23 Completed - Training job completed
Model Artifacts Value
s3://sagemaker-eu-north-1-145023122385/logistic-regrsion-sklearn-custom-2024-10-24-20-04-51-519/output/model.tar.gz


In [21]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "logistic-regression-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role=role,
    entry_point="../src/sagemaker_logistic_regression.py",
    framework_version="0.23-1",
)

In [22]:
model

### Deploy the Model
Once the model is trained, you can deploy it to a real-time endpoint

In [28]:
# Deploy the model to an endpoint
# you can see that model_endpoint is the same as model name
model_endpoint = "logistic-regression-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=model_endpoint
)



INFO:sagemaker:Creating model with name: logistic-regression-sklearn-model-2024-10-24-21-30-17
INFO:sagemaker:Creating endpoint-config with name logistic-regression-sklearn-model-2024-10-24-22-15-11
INFO:sagemaker:Creating endpoint with name logistic-regression-sklearn-model-2024-10-24-22-15-11
      


### Make Predictions using the Deployed Model
After deploying the model, you can use the endpoint to make predictions on new data.

In [ ]:
import numpy as np

# Send the test data to the deployed model for predictions
# prediction only on first two values
predictions = predictor.predict(X_test[0:2].values)
print(predictions)


### Clean up Resources
Once done, it is a good practice to delete the endpoint to avoid ongoing charges.

In [31]:
# predictor.delete_endpoint()
sm_boto3.delete_endpoint(EndpointName=model_endpoint)

hell
